## Load Packages

In [1]:

include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Utility Function

Recall: the CRRA utility function is $U(W) = \frac{W^{1-k}}{1-k}$

In [3]:
U(x,k) =  x.^(1-k)/(1-k)    #CRRA utility function, k is the risk aversion

U (generic function with 1 method)

In [4]:
W = linspace(0.8,1.2,25)


plot(W,U(W,2),color=:red,linewidth=2,legend=false)
title!("Utility function, CRRA")
xlabel!("wealth")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 1.1 
 
 
 -1.2 
 
 
 -1.1 
 
 
 -1.0 
 
 
 -0.9 
 
 
 Utility function, CRRA 
 
 
 wealth 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,372.441 62.4016,351.326 85.4331,331.055 108.465,311.579 131.496,292.853 154.528,274.833 177.559,257.48 200.591,240.759 223.622,224.634 246.654,209.076 
 269.685,194.054 292.717,179.541 315.748,165.512 338.78,151.943 361.811,138.811 384.843,126.097 407.874,113.78 430.906,101.841 453.937,90.2649 476.969,79.034 
 500,68.1334 523.031,57.5488 546.063,47.2666 569.094,37.274 592.126,27.5591 
 "/>

# Expected Utility

Recall: if $\pi_s$ is the probability of outcome $W_s$ and there are $S$ possible outcomes, then

$\text{E}W = \sum\nolimits_{s=1}^{S} \pi_{s}W_{s}$ and 

$\text{E}U(W) = \sum\nolimits_{s=1}^{S} \pi_{s}U(W_{s})$

Concavity of the utility function means that 
$\text{E} U(W) < U(\text{E} W)$

In [5]:
EU(x,π,k) = sum(π.*U(x,k))               #expected utility

EU (generic function with 1 method)

In [6]:
W₁ = 0.85               #possible outcomes
W₂ = 1.15
π₁ = 0.5                #probabilities of outcomes 
π₂ = 1 - π₁

xx = [W₁ U(W₁,2);
      W₂ U(W₂,2)]
println("Possible wealth levels and the corresponding utility")
printmat(xx)

UtilExpW = U(sum([π₁;π₂].*[W₁;W₂]),2)
ExpUtil  = EU([W₁;W₂],[π₁;π₂],2)
printlnPs("Utility at expected value and Expected utility: ",[UtilExpW ExpUtil])

Possible wealth levels and the corresponding utility
     0.850    -1.176
     1.150    -0.870

Utility at expected value and Expected utility:     -1.000    -1.023


In [7]:
plot(W,U(W,2),color=:red,linewidth=2,label="Utility fn")
scatter!([W₁;W₂],U([W₁;W₂],2),color=:green,label="possible outcomes")
scatter!([1.0],[ExpUtil],color=:blue,marker=:rect,label="Expected utility")
plot!(ylims=(-1.3,-0.7))
title!("Utility function, CRRA")
xlabel!("wealth")
ylabel!("")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 1.1 
 
 
 1.2 
 
 
 -1.3 
 
 
 -1.2 
 
 
 -1.1 
 
 
 -1.0 
 
 
 -0.9 
 
 
 -0.8 
 
 
 -0.7 
 
 
 Utility function, CRRA 
 
 
 wealth 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 55.0141,343.701 76.7419,329.037 98.4698,314.961 120.198,301.436 141.925,288.431 163.653,275.917 185.381,263.867 207.109,252.255 228.837,241.057 250.565,230.253 
 272.292,219.821 294.02,209.742 315.748,200 337.476,190.577 359.204,181.458 380.932,172.628 402.659,164.075 424.387,155.784 446.115,147.745 467.843,139.946 
 489.571,132.376 511.298,125.026 533.026,117.885 554.754,110.946 576.482,104.199 
 "/>
 
 
 
 
 
 
 
 
 
 
 Utility fn 
 
 
 
 
 possible outcomes 
 
 
 
 
 Expected utility

# Certainty Equivalent

Recall: the certainty equivalent (here denoted $P$) is the sure (non-stochastic) value that solves 

$U(P) = \text{E}U(W)$

The code below solves for $P$ by a numerial search (actually, we could do it analytically in this case). This is applied to different values of risk aversion.

Notice that the expected wealth level is 1.

In [8]:
using Roots

In [9]:
println("risk aversion and certainly equivalent of process with equal probs of $W₁ and $W₂")
kM = [0.0001;2;5;10;25]
for k in kM
    ceq = fzero(P->EU([W₁;W₂],[π₁;π₂],k) - P^(1-k)/(1-k),[0.5,1.5])
    printlnPs([k ceq])
end

risk aversion and certainly equivalent of process with equal probs of 0.85 and 1.15
     0.000     1.000
     2.000     0.978
     5.000     0.947
    10.000     0.912
    25.000     0.875


# Portfolio Choice with One Risky Asset

Recall: the investor solves $\max\nolimits_{v}\text{E}\ln R_{p}\text{, with }R_{p}=vR_{1} + (1-v)R_{f}$

In the example below, there are two possible outcomes for $R_{1}$ (-0.2 and 0.5) with equal probabilities

In [10]:
function EUc(v,π,Re,Rf)             #expected utility, utility fn is logarithmic
    R = Re + Rf
    Rp = v*R + (1-v)*Rf             #portfolio return
    EU = sum(π .* log.(1+Rp))        #expected utility
    return EU
end

EUc (generic function with 1 method)

In [11]:
v = linspace(-1,1,101)
L = length(v)
EUv = fill(NaN,L)
for i = 1:L
    EUv[i] = EUc(v[i],[0.5;0.5],[-0.3;0.4],0.1)
end    
    
plot(v,EUv,color=:red,linewidth=2,legend=false)
title!("Expected Utility")
xlabel!("weight on risky asset")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 Expected Utility 
 
 
 weight on risky asset 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,372.441 44.8976,361.907 50.4252,351.592 55.9528,341.49 61.4803,331.598 67.0079,321.913 72.5354,312.43 78.063,303.147 83.5906,294.059 89.1181,285.164 
 94.6457,276.459 100.173,267.941 105.701,259.606 111.228,251.453 116.756,243.477 122.283,235.677 127.811,228.05 133.339,220.594 138.866,213.307 144.394,206.185 
 149.921,199.227 155.449,192.431 160.976,185.794 166.504,179.315 172.031,172.992 177.559,166.822 183.087,160.805 188.614,154.938 194.142,149.22 199.669,143.649 
 205.197,138.223 210.724,132.941 216.252,127.802 221.78,122.804 227.307,117.945 232.835,113.226 238.362,108.643 243.89,104.196 249.417,99.8842 254.945,95.7059 
 260.472,91.6602 266,87.746 271.528,83.9624 277.055,80.3083 282.583,76.7829 288.11,73.3852 293.638,70.1144 299.165,66.9697 304.693,63.9503 310.22,61.0554 
 315.748,58.2843 321.276,55.6365 326.803,53.1111 332.331,50.7076 337.858,48.4254 343.386,46.264 348.913,44.2229 354.441,42.3015 359.969,40.4993 365.496,38.8161 
 371.024,37.2513 376.551,35.8046 382.079,34.4756 387.606,33.2641 393.134,32.1698 398.661,31.1923 404.189,30.3315 409.717,29.5872 415.244,28.9591 420.772,28.4472 
 426.299,28.0513 431.827,27.7714 437.354,27.6073 442.882,27.5591 448.409,27.6266 453.937,27.81 459.465,28.1093 464.992,28.5245 470.52,29.0557 476.047,29.7031 
 481.575,30.4669 487.102,31.3471 492.63,32.344 498.157,33.4579 503.685,34.689 509.213,36.0375 514.74,37.5039 520.268,39.0884 525.795,40.7914 531.323,42.6134 
 536.85,44.5547 542.378,46.6159 547.906,48.7974 553.433,51.0997 558.961,53.5235 564.488,56.0693 570.016,58.7376 575.543,61.5292 581.071,64.4448 586.598,67.4851 
 592.126,70.6508 
 "/>

Notice that this problem can be solved by pen and paper, but it 
becomes very difficult when the number of states increase --- and even worse when there are many assets. We therefore apply a numerical optimization algorithm.

In [12]:
using Optim          #minimization routines

In [13]:
#minimize -EU()
Sol = optimize(v-> -EUc(v,[0.5;0.5],[-0.3;0.4],0.1),-1,1)   #try v values between -1 and 1
printlnPs("argmin -EUc(): ",Optim.minimizer(Sol))

println("Compare with the figure")

argmin -EUc():      0.458
Compare with the figure


# Portfolio Choice with Several Risky Assets

This optimization problem has two risky assets and a general CRRA utility function. Numerical optimization is still easy.

In [14]:
function EUe(v,π,R,Rf,γ)
    Rp = v'R + (1-sum(v))*Rf            #portfolio return, row vector (different states)
    Rp = vec(Rp)                        #turn Rp into a column vector
    EU = sum(π .* (1+Rp).^(1-γ)/(1-γ))  #expected utility
    EUNeg = -EU                         #since optimize() minimizes
    return EUNeg
end

EUe (generic function with 1 method)

In [15]:
ROutComes = [-0.03 0.08 0.20;           #2 assets, 3 states
             -0.04 0.22 0.15]           #cell[i,j] is asset i in state j
π = [1/3;1/3;1/3]
Sol = optimize(v->EUe(v,π,ROutComes,0.065,5),[-0.6;1.2])         
printlnPs("optimal weights from max EUe(): ",Optim.minimizer(Sol))

optimal weights from max EUe():     -0.725     1.316


# From Chapter on Mean-Variance Analysis

In [16]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF of risky assets
    n    = length(μ)
    oneV = ones(n,1) 
    Σ_1  = inv(Σ)
    A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
    B    = (μ'Σ_1*oneV)[1]
    C    = (oneV'Σ_1*oneV)[1]
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF
    n   = length(μ)
    μe  = μ - Rf              #excess returns            
    Σ_1 = inv(Σ)
    w   = (mustar-Rf)/(μe'Σ_1*μe)[1] .* Σ_1*μe
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

# Mean-Variance and the Telser Criterion

Let $\mu$ be a vector of expected returns and $\Sigma$ be the covariance matrix of the investible asssets.

The Telser criterion solves the problem

$\max_{v} \mu_{p} \> \text{ subject to} \>  \text{VaR}_{95\%} < 0.1$,

where $\mu_{p} = v'\mu+(1-v)R_f$ is the expected portfolio return.

If the returns are normally distributed then 

$\text{VaR}_{95\%} = -(\mu_p - 1.64\sigma_p)$,

where $\sigma_p = \sqrt{v'\Sigma v}$ is the standard deviation of the portfolio return.

The figure below illustrates that the optimal portfolio is on the CLM (when the returns are normally distributed)

In [17]:
μ = [9; 6]/100                     #means
Σ = [ 256  0;
      0    144]/10000
Rf = 1/100

mustar  = linspace(Rf,0.1,101)
L       = length(mustar)
StdRp   = fill(NaN,L)
StdRpRf = fill(NaN,L)
for i = 1:L
    StdRp[i]   = MVCalc(mustar[i],μ,Σ)[1]
    StdRpRf[i] = MVCalcRf(mustar[i],μ,Σ,Rf)[1]
end   

VaRRestr = -0.1 + 1.64*StdRpRf

println("let's plot it")

let's plot it


In [18]:
plot(StdRpRf*100,mustar*100,color=:blue,linewidth=2,label="CML",xlims=(0,15),ylims=(0,15))
plot!(StdRp*100,mustar*100,color=:red,linewidth=2,label="MVF")
plot!(StdRpRf*100,VaRRestr*100,line=(:dot,2),color=:green,label="VaR restriction")
title!("Mean vs standard deviation (with Rf)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation (with Rf) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:2; stroke-opacity:1; fill:none" points="
 55.1181,349.449 60.0686,347.38 65.0191,345.31 69.9696,343.241 74.9201,341.172 79.8706,339.102 84.8211,337.033 89.7715,334.964 94.722,332.894 99.6725,330.825 
 104.623,328.756 109.574,326.687 114.524,324.617 119.474,322.548 124.425,320.479 129.375,318.409 134.326,316.34 139.276,314.271 144.227,312.202 149.177,310.132 
 154.128,308.063 159.078,305.994 164.029,303.924 168.979,301.855 173.93,299.786 178.88,297.717 183.831,295.647 188.781,293.578 193.732,291.509 198.682,289.439 
 203.633,287.37 208.583,285.301 213.534,283.231 218.484,281.162 223.435,279.093 228.385,277.024 233.336,274.954 238.286,272.885 243.237,270.816 248.187,268.746 
 253.138,266.677 258.088,264.608 263.039,262.539 267.989,260.469 272.94,258.4 277.89,256.331 282.841,254.261 287.791,252.192 292.742,250.123 297.692,248.054 
 302.643,245.984 307.593,243.915 312.544,241.846 317.494,239.776 322.445,237.707 327.395,235.638 332.346,233.569 337.296,231.499 342.247,229.43 347.197,227.361 
 352.148,225.291 357.098,223.222 362.049,221.153 366.999,219.083 371.95,217.014 376.9,214.945 381.85,212.876 386.801,210.806 391.751,208.737 396.702,206.668 
 401.652,204.598 406.603,202.529 411.553,200.46 416.504,198.391 421.454,196.321 426.405,194.252 431.355,192.183 436.306,190.113 441.256,188.044 446.207,185.975 
 451.157,183.906 456.108,181.836 461.058,179.767 466.009,177.698 470.959,175.628 475.91,173.559 480.86,171.49 485.811,169.42 490.761,167.351 495.712,165.282 
 500.662,163.213 505.613,161.143 510.563,159.074 515.514,157.005 520.464,154.935 525.415,152.866 530.365,150.797 535.316,148.728 540.266,146.658 545.217,144.589 
 550.167,142.52 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 1546.38,349.449 1525.48,347.38 1504.61,345.31 1483.75,343.241 1462.91,341.172 1442.09,339.102 1421.29,337.033 1400.51,334.964 1379.75,332.894 1359.02,330.825 
 1338.31,328.756 1317.63,326.687 1296.97,324.617 1276.35,322.548 1255.75,320.479 1235.18,318.409 1214.65,316.34 1194.16,314.271 1173.69,312.202 1153.27,310.132 
 1132.89,308.063 1112.56,305.994 1092.27,303.924 1072.02,301.855 1051.83,299.786 1031.7,297.717 1011.62,295.647 991.609,293.578 971.661,291.509 951.784,289.439 
 931.983,287.37 912.263,285.301 892.629,283.231 873.089,281.162 853.647,279.093 834.313,277.024 815.094,274.954 796,272.885 777.039,270.816 758.224,268.746 
 739.565,266.677 721.076,264.608 702.772,262.539 684.669,260.469 666.785,258.4 649.138,256.331 631.752,254.261 614.651,252.192 597.861,250.123 581.412,248.054 
 565.337,245.984 549.672,243.915 534.459,241.846 519.741,239.776 505.567,237.707 491.989,235.638 479.065,233.569 466.857,231.499 455.43,229.43 444.853,227.361 
 435.196,225.291 426.532,223.222 418.932,221.153 412.462,219.083 407.187,217.014 403.159,214.945 400.424,212.876 399.01,210.806 398.936,208.737 400.201,206.668 
 402.791,204.598 406.677,202.529 411.816,200.46 418.156,198.391 425.634,196.321 434.183,194.252 443.733,192.183 454.211,190.113 465.548,188.044 477.672,185.975 
 490.519,183.906 504.027,181.836 518.138,179.767 532.798,177.698 547.959,175.628 563.575,173.559 579.606,171.49 596.015,169.42 612.769,167.351 629.838,165.282 
 647.193,163.213 664.812,161.143 682.671,159.074 700.751,157.005 719.033,154.935 737.502,152.866 756.142,150.797 774.941,148.728 793.886,146.658 812.966,144.589 
 832.172,142.52 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="1, 2" points="
 55.1181,602.362 60.0686,597.148 65.0191